Name: Lorenzo Ausiello

Date: 02/19/2024

In [1]:
import numpy as np
import yfinance
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
# Set seed of random number generator
CWID = 20021869 
personal = CWID % 10000
np.random.seed(personal)

C:\Users\fdimino\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Question 1 (5pt)

### Question 1.1
Use the `yfinance` package (or other method of your choice) to obtain the daily adjusted close prices for `SPY` and `IEF`.  You should have at least 5 years of data for both assets. Do **not** include any data after January 1, 2023.  You should inspect the dates for your data to make sure you are including everything appropriately.  Create a binary variable whether the `SPY` returns are above the `IEF` returns on a each day. Create a data frame (or array) of the daily log returns both both stocks along with the lagged returns (at least 2 lags) and your binary class variable.  Use the `print` command to display your data.

In [2]:
import yfinance as yf
from datetime import datetime
import pandas as pd

# Download historical data for SPY and IEF
start_date = datetime(2017, 1, 1)
end_date = datetime(2023, 1, 1)

myData = yf.download(['SPY', 'IEF'], start=start_date, end=end_date)
SPY = myData['Adj Close']['SPY']
IEF = myData['Adj Close']['IEF']

# Calculate daily log returns
rSPY = (np.log(SPY) - np.log(SPY.shift(1))).dropna()
rIEF = (np.log(IEF) - np.log(IEF.shift(1))).dropna()

df = pd.DataFrame({'SPY':rSPY, 'IEF': rIEF})

for i in range(1, 4):
    df[f"SPY_lag{i}"] = df["SPY"].shift(i)
    df[f"IEF_lag{i}"] = df["IEF"].shift(i)
    
df= df.dropna()

df['SPY>IEF'] = (df['SPY'] > df['IEF']).astype(int)

print(df)

[*********************100%%**********************]  2 of 2 completed

                 SPY       IEF  SPY_lag1  IEF_lag1  SPY_lag2  IEF_lag2  \
Date                                                                     
2017-01-09 -0.003306  0.003799  0.003571 -0.004558 -0.000795  0.006462   
2017-01-10  0.000000 -0.000474 -0.003306  0.003799  0.003571 -0.004558   
2017-01-11  0.002822  0.001137  0.000000 -0.000474 -0.003306  0.003799   
2017-01-12 -0.002513  0.000568  0.002822  0.001137  0.000000 -0.000474   
2017-01-13  0.002293 -0.002180 -0.002513  0.000568  0.002822  0.001137   
...              ...       ...       ...       ...       ...       ...   
2022-12-23  0.005736 -0.004538 -0.014369 -0.000308  0.014842  0.001235   
2022-12-27 -0.003951 -0.008407  0.005736 -0.004538 -0.014369 -0.000308   
2022-12-28 -0.012506 -0.002400 -0.003951 -0.008407  0.005736 -0.004538   
2022-12-29  0.017840  0.004899 -0.012506 -0.002400 -0.003951 -0.008407   
2022-12-30 -0.002638 -0.004168  0.017840  0.004899 -0.012506 -0.002400   

            SPY_lag3  IEF_lag3  SPY>I

### Question 1.2
Split your data into training and testing sets (80% training and 20% test). This split should be done so that the causal relationship is kept consistent (i.e., split data at a specific time).

Run a logistic regression of the binary variable (of `SPY` returns greater than `IEF` returns) as a function of the lagged returns (at least 2 lags) for both stocks.
This should be of the form (assuming 2 lags) of $p_{t} = [1 + \exp(-[\beta_0 + \beta_{SPY,1} r_{SPY,t-1} + \beta_{SPY,2} r_{SPY,t-2} + \beta_{SPY,3} r_{SPY,t-3} + \beta_{IEF,1} r_{IEF,t-1} + \beta_{IEF,2} r_{IEF,t-2} + \beta_{IEF,3} r_{IEF,t-3}])]^{-1}$.
Evaluate the performance of this model by printing the confusion matrix and accuracy on the test data.

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_auc_score

# Split into training and testing
data_length = len(df)
train_size = int(data_length * 0.8)
test_size = data_length - train_size

train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

# Define features and target variable for training and testing sets
X_train = train_data[['SPY_lag1', 'SPY_lag2', 'SPY_lag3', 'IEF_lag1', 'IEF_lag2', 'IEF_lag3']]
y_train = train_data['SPY>IEF']

X_test = test_data[['SPY_lag1', 'SPY_lag2', 'SPY_lag3', 'IEF_lag1', 'IEF_lag2', 'IEF_lag3']]
y_test = test_data['SPY>IEF']

# Fit logistic regression model on scaled data
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

# Predict on scaled test data
y_pred = log_reg.predict(X_test)

# Evaluate the model
conf_matrix = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc_score = roc_auc_score(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)
print("Accuracy:", accuracy)
print("F1:", f1)
print('roc_auc_score:', roc_auc_score)

Confusion Matrix:
[[  0 146]
 [  0 156]]
Accuracy: 0.5165562913907285
F1: 0.6812227074235807
roc_auc_score: 0.5


<font color='red'>Solution:</font> it predicts just one class (1) with accuracy 51.65%. F1 score indicates the model has a good equilibrium between precision and recall. Roc Auc score says the model is not able to distinguish between positive and negative classes.

## Question 2 (20pt)

### Question 2.1
Using the same data, train/test split ratio, and consider the same classification problem as in Question 1.2.
Create a (plain) recurrent neural network of your own design using a time step of 3.
You may choose any activation functions you wish.

In [4]:
import tensorflow as tf
from tensorflow import keras

# Construct neural network
# Need to reshape for use in RNN (fa una sorte di expandign window)
def get_XY(dat, time_steps):
    # Indices of target array
    Y_ind = np.arange(time_steps, len(dat), time_steps)
    Y = dat['SPY>IEF'][Y_ind]
    # Prepare X
    rows_x = len(Y)
    X = dat[['SPY_lag1', 'IEF_lag1']].iloc[range(time_steps*rows_x)]
    X_array = X.values
    X = np.reshape(X_array, (rows_x, time_steps, 2)) 
    return X, Y
 
time_steps = 3 #similar to kernel size for TDNN.
train_X, train_y = get_XY(train_data, time_steps)
test_X, test_y = get_XY(test_data, time_steps)

RNN = keras.Sequential()
RNN.add(keras.layers.Input(shape=(3,2))) #shape = (time_steps,features)
RNN.add(keras.layers.SimpleRNN(32, return_sequences=True, activation='relu'))
# add return_sequences=True to all RNN layers except the last one to allow for stacking
# Quando return_sequences=True è impostato, lo strato RNN restituisce un tensore per ogni passaggio temporale, il che significa che l'output sarà tridimensionale, con le dimensioni (batch_size, time_steps, output_features). Ciò consente di collegare direttamente uno strato RNN a un altro strato RNN successivo.

RNN.add(keras.layers.SimpleRNN(32, activation='relu'))
# Tuttavia, nell'ultimo strato RNN, di solito si imposta return_sequences=False (o si omette completamente) perché l'output dell'ultimo strato RNN deve essere un tensore bidimensionale, 

RNN.add(keras.layers.Dense(32, activation='relu')) # spetta a noi se metterlo o meno ma e` buono dato che vogliamo riportare al dense finale
RNN.add(keras.layers.Dense(1, activation='sigmoid')) #1 output y

# State the loss function and optimizer
RNN.compile(optimizer=keras.optimizers.Adam(),loss='binary_crossentropy', metrics=['accuracy'])

### Question 2.2
Train this neural network on the training data.  
Evaluate the performance of this model by printing the confusion matrix and accuracy on the test data.

In [5]:
# Fit the model to training data
history = RNN.fit(train_X,train_y,epochs=100,validation_split=0.1,verbose=0)

# Predictions on the test set
y_pred_prob = RNN.predict(test_X)
y_pred_binary = (y_pred_prob > 0.5).astype(int)

# Confusion matrix and accuracy
conf_matrix = confusion_matrix(test_y, y_pred_binary)
accuracy = accuracy_score(test_y, y_pred_binary)
f1 = f1_score(test_y, y_pred_binary)
print("Confusion Matrix:")
print(conf_matrix)
print("Accuracy:", accuracy)
print("F1:", f1)



4/4 [==============================] - 0s 5ms/step
Confusion Matrix:
[[14 29]
 [21 36]]
Accuracy: 0.5
F1: 0.5901639344262295


<font color='red'>Solution:</font> it predicts both classes but the accuracy and F1 score are slightly worse

## Question 3 (20pt)

### Question 3.1
Using the same data, train/test split ratio, and consider the same classification problem as in Question 1.2.
Create a long short-term memory (LSTM) network of your own design using a time step of 3.
You may choose any activation functions you wish.

In [6]:
HIDDEN = 16
LSTM = keras.Sequential()
LSTM.add(keras.layers.Input(shape=(time_steps,2))) #shape = (time_steps,features)
LSTM.add(keras.layers.LSTM(HIDDEN, return_sequences=True, activation='relu'))
LSTM.add(keras.layers.LSTM(HIDDEN, activation='relu'))
# qua non mettiamo dense dato che abbiamo gia molti parametri.
LSTM.add(keras.layers.Dense(1, activation='sigmoid')) #1 output y

# State the loss function and optimizer 
LSTM.compile(optimizer=keras.optimizers.Adam(),loss='binary_crossentropy', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 3, 16)             1216      
                                                                 
 lstm_1 (LSTM)               (None, 16)                2112      
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 3345 (13.07 KB)
Trainable params: 3345 (13.07 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Question 3.2
Train this neural network on the training data.
Evaluate the performance of this model by printing the confusion matrix and accuracy on the test data.

In [8]:
# Fit the model to training data
history = LSTM.fit(train_X,train_y,epochs=100,validation_split=0.1,verbose=0)

# Predictions on the test set
y_pred_prob = LSTM.predict(test_X)
y_pred_binary = (y_pred_prob > 0.5).astype(int)

# Confusion matrix and accuracy
conf_matrix = confusion_matrix(test_y, y_pred_binary)
accuracy = accuracy_score(test_y, y_pred_binary)
f1 = f1_score(test_y, y_pred_binary)
print("Confusion Matrix:")
print(conf_matrix)
print("Accuracy:", accuracy)
print("F1:", f1)

4/4 [==============================] - 0s 4ms/step
Confusion Matrix:
[[ 0 43]
 [ 0 57]]
Accuracy: 0.57
F1: 0.7261146496815287


<font color='red'>Solution:</font> we have the best accuracy and f1 score, but it just predicts one class

## Question 4 (20pt)

## Question 4.1
Consider the same classification problem as in Question 1.2.
Of the methods considered in this assignment, which would you recommend in practice?
Explain briefly (1 paragraph) why you choose this fit. 

<font color='red'>Solution:</font> I'd recommend the LSTM because it has the best accuracy and F1 score. However, one can argue that it just predicts one class

## Question 4.2
Recreate your data set using data from January 1, 2023 through December 31, 2023.
Using the method your would implement in practice, invest in the asset (``SPY`` or ``IEF``) depending on your predictions.
Print the returns your portfolio would obtain from following this strategy. Comment on how this portfolio compares with the ``SPY`` and ``IEF`` returns and risks.

In [9]:
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 31)

myData = yf.download(['SPY', 'IEF'], start=start_date, end=end_date)
SPY = myData['Adj Close']['SPY']
IEF = myData['Adj Close']['IEF']

# Calculate daily log returns
rSPY = (np.log(SPY) - np.log(SPY.shift(1))).dropna()
rIEF = (np.log(IEF) - np.log(IEF.shift(1))).dropna()

df = pd.DataFrame({'SPY': rSPY, 'IEF': rIEF})

for i in range(1, 4):
    df[f"SPY_lag{i}"] = df["SPY"].shift(i)
    df[f"IEF_lag{i}"] = df["IEF"].shift(i)

df = df.dropna()

df['SPY>IEF'] = (df['SPY'] > df['IEF']).astype(int)

def get_XY(dat, time_steps):
    # Indices of target array
    Y_ind = np.arange(time_steps, len(dat), time_steps)
    Y = dat['SPY>IEF'][Y_ind]
    # Prepare X
    rows_x = len(Y)
    X = dat[['SPY_lag1', 'IEF_lag1']].iloc[range(time_steps*rows_x)]
    X_array = X.values
    X = np.reshape(X_array, (rows_x, time_steps, 2)) 
    return X, Y
 
time_steps = 3 
test_X, test_y = get_XY(df, time_steps)

# Make predictions
y_pred_prob = LSTM.predict(test_X)
y_pred_binary = (y_pred_prob > 0.5).astype(int)

[*********************100%%**********************]  2 of 2 completed

3/3 [==============================] - 0s 3ms/step


In [22]:
# rebalance
y_pred_repeated = np.repeat(y_pred_binary[:-1], 3)
y_pred_rebalanced = np.concatenate((y_pred_repeated, np.repeat(y_pred_binary[-1], 6)))

In [23]:
df['Predictions'] = y_pred_rebalanced

In [25]:
df['Portfolio_Returns'] = np.where(df['Predictions'] == 1, df['SPY'], df['IEF'])

In [26]:
print(df['Portfolio_Returns'])

Date
2023-01-09   -0.000567
2023-01-10    0.006988
2023-01-11    0.012569
2023-01-12    0.003634
2023-01-13    0.003872
                ...   
2023-12-22    0.002008
2023-12-26    0.004214
2023-12-27    0.001806
2023-12-28    0.000378
2023-12-29   -0.002899
Name: Portfolio_Returns, Length: 246, dtype: float64


In [29]:
mean_p = df['Portfolio_Returns'].mean()
sd_p = df['Portfolio_Returns'].std()

print(mean_p)
print(sd_p)

# SPY
mean_s = df['SPY'].mean()
sd_s = df['SPY'].std()
print('Spy mean :', mean_s)
print('Spy sd:', sd_s)

# IEF
mean = df['IEF'].mean()
sd = df['IEF'].std()
print('Ief mean :', mean)
print('Ief sd:', sd)

# portfolio
sharpe_ratio = (mean_p-mean)/sd_p
print('sharpe ratio portfolio:', sharpe_ratio)

0.0008855287850654267
0.00812326478293696
Spy mean : 0.0008855287850654267
Spy sd: 0.00812326478293696
Ief mean : 3.663575315225e-05
Ief sd: 0.00586987043002503
sharpe ratio portfolio: 0.10450146026217061


<font color='red'>Solution:</font> since we just predict SPY, the portfolio returns and volatility are the same as SPY. SD is slightly higher than IEF, but we have better returns